# Time Series Analysis
## Chapter 5: A Dog on a Leash?

### A Dog on a Leash? (Part 1)
The Heating Oil and Natural Gas prices are pre-loaded in DataFrames `HO` and `NG`. We will plot both price series, which look like random walks. Then, we will plot the difference (spread) between the two series, which should look more like a mean-reverting series. To put both series in the same units, we multiply Heating Oil prices (in $/gallon) by 7.25 to convert it to $/millionBTU, the same unit as Natural Gas.  The data for continuous futures (each contract has to be spliced together in a continuous series as contracts expire) was obtained from Quandl.(https://blog.quandl.com/api-for-futures-data)


In [ ]:
import matplotlib.pyplot as plt

# Plot the prices separately
plt.figure(figsize=(10,6))
plt.subplot(2,1,1)
plt.plot(7.25*HO, label='Heating Oil')
plt.plot(NG, label='Natural Gas')
plt.legend(loc='best', fontsize='small')

# Plot the spread
plt.subplot(2,1,2)
plt.plot(7.25*HO - NG, label='Spread')
plt.legend(loc='best', fontsize='small')
plt.axhline(y=0, linestyle='--', color='k')
plt.show()

### A Dog on a Leash? (Part 2)
To verify that Heating Oil and Natural Gas prices are cointegrated, we will apply the Augmented Dickey-Fuller (ADF) test separately to each series and then to the spread to check for stationarity.

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Compute the ADF for HO and NG
result_HO = adfuller(HO['Close'])
print('The p-value for the ADF test on HO is', result_HO[1])

result_NG = adfuller(NG['Close'])
print('The p-value for the ADF test on NG is', result_NG[1])

# Compute the ADF of the spread
result_spread = adfuller(7.25 * HO['Close'] - NG['Close'])
print('The p-value for the ADF test on the spread is', result_spread[1])

### Are Bitcoin and Ethereum Cointegrated?
Cointegration involves two steps: regressing one time series on the other to get the cointegration vector, and then perform an ADF test on the residuals of the regression. In the last example, there was no need to perform the first step since we implicitly assumed the cointegration vector was . In other words, we took the difference between the two series (after doing a units conversion). Here, you will do both steps.

You will regress the value of one cryptocurrency, bitcoin (BTC), on another cryptocurrency, ethereum (ETH). If we call the regression coefficient , then the cointegration vector is simply . Then perform the ADF test on BTC  ETH. Bitcoin and Ethereum prices are pre-loaded in DataFrames BTC and ETH.


In [ ]:
import statsmodels.api as sm

# Regress BTC on ETH
ETH = sm.add_constant(ETH)
result = sm.OLS(BTC, ETH).fit()

# Compute ADF
b = result.params[1]
adf_stats = adfuller(BTC['Price'] - b*ETH['Price'])
print('The p-value for the ADF test is', adf_stats[1])

### Is Temperature a Random Walk (with Drift)?
An ARMA model is a simplistic approach to forecasting climate changes, but it illustrates many of the topics covered in this class.

The DataFrame temp_NY contains the average annual temperature in Central Park, NY from 1870-2016 (the data was downloaded from the NOAA here). Plot the data and test whether it follows a random walk (with drift).


In [ ]:
from statsmodels.tsa.stattools import adfuller
import pandas as pd

# Convert the index to a datetime object
temp_NY.index = pd.to_datetime(temp_NY.index, format='%Y')

# Plot average temperatures
temp_NY.plot()
plt.show()

# Compute and print ADF p-value
result = adfuller(temp_NY['TAVG'])
print('The p-value for the ADF test is', result[1])

### Getting 'Warmed' Up: Look at Autocorrelations
Since the temperature series, temp_NY, is a random walk with drift, take first differences to make it stationary. Then compute the sample ACF and PACF. This will provide some guidance on the order of the model.


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Take first difference of the temperature Series
chg_temp = temp_NY.diff().dropna()

# Plot the ACF and PACF
fig, axes = plt.subplots(2,1, figsize=(10,6))
plot_acf(chg_temp, lags=20, ax=axes[0])
plot_pacf(chg_temp, lags=20, ax=axes[1])
plt.show()

### Which ARMA Model is Best?
Recall from Chapter 3 that the Akaike Information Criterion (AIC) can be used to compare models with different numbers of parameters. It measures goodness-of-fit, but places a penalty on models with more parameters to discourage overfitting. Lower AIC scores are better.

Fit the temperature data to an AR(1), AR(2), and ARMA(1,1) and see which model is the best fit, using the AIC criterion. The AR(2) and ARMA(1,1) models have one more parameter than the AR(1) has.

The annual change in temperature is in a DataFrame chg_temp.

For each ARMA model, create an instance of the ARMA class, passing the data and the order=(p,q). p is the autoregressive order; q is the moving average order.
Fit the model using the method .fit().
Print the AIC value, found in the .aic element of the results.

In [ ]:
from statsmodels.tsa.arima_model import ARMA

# Fit AR(1) model
mod_ar1 = ARMA(chg_temp, order=(1,0))
res_ar1 = mod_ar1.fit()
print('The AIC for an AR(1) is:', res_ar1.aic)

# Fit AR(2) model
mod_ar2 = ARMA(chg_temp, order=(2,0))
res_ar2 = mod_ar2.fit()
print('The AIC for an AR(2) is:', res_ar2.aic)

# Fit ARMA(1,1) model
mod_arma11 = ARMA(chg_temp, order=(1,1))
res_arma11 = mod_arma11.fit()
print('The AIC for an ARMA(1,1) is:', res_arma11.aic)

### Don't Throw Out That Winter Coat Yet
Finally, you will forecast the temperature over the next 30 years using an ARMA(1,1) model, including confidence bands around that estimate. Keep in mind that the estimate of the drift will have a much bigger impact on long range forecasts than the ARMA parameters.

Earlier, you determined that the temperature data follows a random walk and you looked at first differencing the data. In this exercise, you will use the ARIMA module on the temperature data (before differencing), which is identical to using the ARMA module on changes in temperature, followed by taking cumulative sums of these changes to get the temperature forecast.

The data is preloaded in a DataFrame called temp_NY.

Create an instance of the ARIMA class called mod for an integrated ARMA(1,1) model
The d in order(p,d,q) is one, since we first differenced once
Fit mod using the .fit() method and call the results res
Forecast the series using the plot_predict() method on res
Choose the start date as 1872-01-01 and the end date as 2046-01-01

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

# Forecast temperatures using ARIMA(1,1,1)
mod = ARIMA(temp_NY, order=(1,1,1))
res = mod.fit()

# Plot the forecast
res.plot_predict(start='1872-01-01', end='2046-01-01')
plt.show()